<a href="https://colab.research.google.com/github/ymoslem/OpenNMT-Tutorial/blob/main/2-NMT-Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install OpenNMT-py 3.x
!pip3 install OpenNMT-py==3.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.5/250.5 kB 6.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 36.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 67.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 82.7 MB/s eta 0:00:00:00:01


# Prepare Your Datasets
Please make sure you have completed the [first exercise](https://colab.research.google.com/drive/1rsFPnAQu9-_A6e2Aw9JYK3C8mXx9djsF?usp=sharing).

In [2]:
# Open the folder where you saved your prepapred datasets from the first exercise
# You might need to mount your Google Drive first
# %cd /content/drive/MyDrive/nmt/
# !ls

# Create the Training Configuration File

The following config file matches most of the recommended values for the Transformer model [Vaswani et al., 2017](https://arxiv.org/abs/1706.03762). As the current dataset is small, we reduced the following values: 
* `train_steps` - for datasets with a few millions of sentences, consider using a value between 100000 and 200000, or more! Enabling the option `early_stopping` can help stop the training when there is no considerable improvement.
* `valid_steps` - 10000 can be good if the value `train_steps` is big enough. 
* `warmup_steps` - obviously, its value must be less than `train_steps`. Try 4000 and 8000 values.

Refer to [OpenNMT-py training parameters](https://opennmt.net/OpenNMT-py/options/train.html) for more details. If you are interested in further explanation of the Transformer model, you can check this article, [Illustrated Transformer](https://jalammar.github.io/illustrated-transformer/).

In [3]:
# Create the YAML configuration file
# On a regular machine, you can create it manually or with nano
# Note here we are using some smaller values because the dataset is small
# For larger datasets, consider increasing: train_steps, valid_steps, warmup_steps, save_checkpoint_steps, keep_checkpoint

config = '''# config.yaml


## Where the samples will be written
save_data: run

# Training files
data:
    corpus_1:
        path_src: /kaggle/input/nmt-data-subword/split_train2023_10kto100k.vi.subword
        path_tgt: /kaggle/input/nmt-data-subword/split_train2023_10kto100k.lo.subword
        transforms: [filtertoolong]
    valid:
        path_src: /kaggle/input/nmt-data-subword/split_val2023_10kto100k.vi.subword
        path_tgt: /kaggle/input/nmt-data-subword/split_val2023_10kto100k.lo.subword
        transforms: [filtertoolong]

# Vocabulary files, generated by onmt_build_vocab
src_vocab: run/source.vocab
tgt_vocab: run/target.vocab

# Vocabulary size - should be the same as in sentence piece
src_vocab_size: 50000
tgt_vocab_size: 50000

# Filter out source/target longer than n if [filtertoolong] enabled
src_seq_length: 150
src_seq_length: 150

# Tokenization options
src_subword_model: /kaggle/input/nmt-data-subword/source.model
tgt_subword_model: /kaggle/input/nmt-data-subword/target.model

# Where to save the log file and the output models/checkpoints
log_file: train.log
save_model: models/model.vilo

# Stop training if it does not imporve after n validations
early_stopping: 4

# Default: 5000 - Save a model checkpoint for each n
save_checkpoint_steps: 5000

# To save space, limit checkpoints to last n
# keep_checkpoint: 3

seed: 3435

# Default: 100000 - Train the model to max n steps 
# Increase to 200000 or more for large datasets
# For fine-tuning, add up the required steps to the original steps
train_steps: 100000

# Default: 10000 - Run validation after n steps
valid_steps: 1000

# Default: 4000 - for large datasets, try up to 8000
warmup_steps: 1000
report_every: 100

# Number of GPUs, and IDs of GPUs
world_size: 1
gpu_ranks: [0]

# Batching
bucket_size: 262144
num_workers: 0  # Default: 2, set to 0 when RAM out of memory
batch_type: "tokens"
batch_size: 4096   # Tokens per batch, change when CUDA out of memory
valid_batch_size: 2048
max_generator_batches: 2
accum_count: [4]
accum_steps: [0]

# Optimization
model_dtype: "fp16"
optim: "adam"
learning_rate: 2
# warmup_steps: 8000
decay_method: "noam"
adam_beta2: 0.998
max_grad_norm: 0
label_smoothing: 0.1
param_init: 0
param_init_glorot: true
normalization: "tokens"

# Model
encoder_type: transformer
decoder_type: transformer
position_encoding: true
enc_layers: 6
dec_layers: 6
heads: 8
hidden_size: 512
word_vec_size: 512
transformer_ff: 2048
dropout_steps: [0]
dropout: [0.1]
attention_dropout: [0.1]
'''

with open("config.yaml", "w+") as config_yaml:
  config_yaml.write(config)

In [4]:
# [Optional] Check the content of the configuration file
!cat config.yaml

# config.yaml


## Where the samples will be written
save_data: run

# Training files
data:
    corpus_1:
        path_src: /kaggle/input/nmt-data-subword/split_train2023_10kto100k.vi.subword
        path_tgt: /kaggle/input/nmt-data-subword/split_train2023_10kto100k.lo.subword
        transforms: [filtertoolong]
    valid:
        path_src: /kaggle/input/nmt-data-subword/split_val2023_10kto100k.vi.subword
        path_tgt: /kaggle/input/nmt-data-subword/split_val2023_10kto100k.lo.subword
        transforms: [filtertoolong]

# Vocabulary files, generated by onmt_build_vocab
src_vocab: run/source.vocab
tgt_vocab: run/target.vocab

# Vocabulary size - should be the same as in sentence piece
src_vocab_size: 50000
tgt_vocab_size: 50000

# Filter out source/target longer than n if [filtertoolong] enabled
src_seq_length: 150
src_seq_length: 150

# Tokenization options
src_subword_model: /kaggle/input/nmt-data-subword/source.model
tgt_subword_model: /kaggle/input/nmt-data-subword/target.model


# Build Vocabulary

For large datasets, it is not feasable to use all words/tokens found in the corpus. Instead, a specific set of vocabulary is extracted from the training dataset, usually betweeen 32k and 100k words. This is the main purpose of the vocabulary building step.

In [5]:
# Find the number of CPUs/cores on the machine
!nproc --all

4


In [6]:
# Build Vocabulary

# -config: path to your config.yaml file
# -n_sample: use -1 to build vocabulary on all the segment in the training dataset
# -num_threads: change it to match the number of CPUs to run it faster

!onmt_build_vocab -config config.yaml -n_sample -1 -num_threads 4

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[2023-11-08 06:39:48,531 INFO] Counter vocab from -1 samples.
[2023-11-08 06:39:48,531 INFO] n_sample=-1: Build vocab on full datasets.
[2023-11-08 06:39:50,840 INFO] * Transform statistics for corpus_1(25.00%):
			* FilterTooLongStats(filtered=409)

[2023-11-08 06:39:50,844 INFO] * Transform statistics for corpus_1(25.00%):
			* FilterTooLongStats(filtered=440)

[2023-11-08 06:39:50,846 INFO] * Transform statistics for corpus_1(25.00%):
			* FilterTooLongStats(filtered=461)

[2023-11-08 06:39:50,848 INFO] * Transform statistics for corpus_1(25.00%):
			* FilterTooLongStats(filtered=384)

[2023-11-08 06:39:50,989 INFO] Counters src: 12288
[2023-11-08 06:39:50,990 INFO] Counters tgt: 27900


From the **Runtime menu** > **Change runtime type**, make sure that the "**Hardware accelerator**" is "**GPU**".


In [7]:
# Check if the GPU is active
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-ce52e806-e7e0-68d5-bbb2-24ed59359682)


In [8]:
# Check if the GPU is visable to PyTorch

import torch

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

gpu_memory = torch.cuda.mem_get_info(0)
print("Free GPU memory:", gpu_memory[0]/1024**2, "out of:", gpu_memory[1]/1024**2)

True
Tesla P100-PCIE-16GB
Free GPU memory: 15665.75 out of: 16280.875


# Training

Now, start training your NMT model! 🎉 🎉 🎉

In [9]:
!rm -rf /kaggle/working/nmt/models/

In [10]:
# Train the NMT model
!onmt_train -config config.yaml

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[2023-11-08 06:40:08,149 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2023-11-08 06:40:08,150 INFO] Parsed 2 corpora from -data.
[2023-11-08 06:40:08,150 INFO] Get special vocabs from Transforms: {'src': [], 'tgt': []}.
[2023-11-08 06:40:08,345 INFO] The first 10 tokens of the vocabs are:['<unk>', '<blank>', '<s>', '</s>', '▁', ',', '0', '.', '1', '2']
[2023-11-08 06:40:08,345 INFO] The decoder start token is: <s>
[2023-11-08 06:40:08,345 INFO] Building model...
[2023-11-08 06:40:09,407 INFO] Switching model to float32 for amp/apex_amp
[2023-11-08 06:40:09,407 INFO] Non quantized layer compute is fp16
^C


In [11]:
# For error debugging try:
# !dmesg -T

# Translation

Translation Options:
* `-model` - specify the last model checkpoint name; try testing the quality of multiple checkpoints
* `-src` - the subworded test dataset, source file
* `-output` - give any file name to the new translation output file
* `-gpu` - GPU ID, usually 0 if you have one GPU. Otherwise, it will translate on CPU, which would be slower.
* `-min_length` - [optional] to avoid empty translations
* `-verbose` - [optional] if you want to print translations

Refer to [OpenNMT-py translation options](https://opennmt.net/OpenNMT-py/options/translate.html) for more details.

In [23]:
!git clone https://github.com/ymoslem/MT-Preparation.git

fatal: destination path 'MT-Preparation' already exists and is not an empty directory.


In [24]:
# Translate the "subworded" source file of the test dataset
# Change the model name, if needed.
!onmt_translate -model models/model.vilo_step_100000.pt -src /kaggle/input/nmt-data-subword/split_val2023_10kto100k.vi.subword -output UN.lo.translated -gpu 0 -min_length 1

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[2023-11-08 06:41:15,967 INFO] Loading checkpoint from /kaggle/input/result-model-nmt/model.fren_step_3000.pt
[2023-11-08 06:41:23,487 INFO] Loading data into the model
[2023-11-08 06:49:22,274 INFO] PRED SCORE: -0.4574, PRED PPL: 1.58 NB SENTENCES: 12000


In [25]:
# Check the first 5 lines of the translation file
!head -n 5 UN.lo.translated

▁ພ້ອມກັນ ນັ້ນ , ▁ ກະຊວງກະສິກໍາ ▁ແລະ ▁ພັດທະນາຊົນນະບົດ ▁ຫວຽດນາມ ▁ກໍ່ ເພີ່ມທະວີ ວຽກງານ ຈັດຕັ້ງ ການຮັບ ທາ ບ , ▁ພິເສດແມ່ນ ▁ພາຍ ▁ 2 ▁ປະເທດ , ▁ພິເສດແມ່ນ ພະຍຸ , ▁ ປ້ອງກັນ , ▁ຕ້ານ ບັນດາ ຂໍ້ກໍານົດ , ▁ຕ້ານ ເຊື້ອພະຍາດ ຕິດຕໍ່ , ▁ພິເສດແມ່ນ ວິສາຫະກິດ ...
▁ຈາກ ບໍລິສັດ ແຟັກ : ▁+ ▁ 8 6 - 5 2 3 .
▁ຂ້າພະ ▁ເຈົ້າ ອະທິຖານ ▁ວ່າ ▁ໃນ ▁ໂລກ ▁ນີ້ ▁ຈະ ▁ບັນລຸ ▁ຄວາມ ▁ສໍາ ▁ເລັດ , ▁ເຮົາ ▁ຈະ ▁ໄດ້ ▁ຮັບ ▁ຄວາມ ▁ຖ່ອມ ▁ຕົນ ▁ເພື່ອ ▁ຈະ ▁ໄດ້ ▁ຮັບ .
▁ໃນຄວາມເປັນຈິງ , ▁ສັງຄົມ ໄດ້ ຂ້າຕົວ ຕາຍ .
▁ 1 2 ▁ບັດ ▁ນີ້ ▁ຫລາຍ ▁ຄົນ ▁ບໍ່ ▁ໄດ້ ▁ຕາຍ ▁ໄປ , ▁ແຕ່ ▁ລູກ ▁ກັບ ▁ເມຍ ▁ຂອງ ▁ຕົນ ▁ອີກ , ▁ແຕ່ ▁ເຂົາ ▁ໄດ້ ▁ຕາຍ ▁ໄປ ▁ດ້ວຍ ▁ຄວາມ ▁ຕາຍ .


In [27]:
# If needed install/update sentencepiece
!pip3 install --upgrade -q sentencepiece

# Desubword the translation file
!python3 MT-Preparation/subwording/3-desubword.py /kaggle/input/nmt-data-subword/target.model UN.lo.translated

Done desubwording! Output: UN.lo.translated.desubword


In [28]:
# Check the first 5 lines of the desubworded translation file
!head -n 5 UN.lo.translated.desubword

ພ້ອມກັນນັ້ນ, ກະຊວງກະສິກໍາ ແລະ ພັດທະນາຊົນນະບົດ ຫວຽດນາມ ກໍ່ເພີ່ມທະວີວຽກງານຈັດຕັ້ງການຮັບທາບ, ພິເສດແມ່ນ ພາຍ 2 ປະເທດ, ພິເສດແມ່ນພະຍຸ, ປ້ອງກັນ, ຕ້ານບັນດາຂໍ້ກໍານົດ, ຕ້ານເຊື້ອພະຍາດຕິດຕໍ່, ພິເສດແມ່ນວິສາຫະກິດ...
ຈາກບໍລິສັດແຟັກ: + 86-523.
ຂ້າພະ ເຈົ້າອະທິຖານ ວ່າ ໃນ ໂລກ ນີ້ ຈະ ບັນລຸ ຄວາມ ສໍາ ເລັດ, ເຮົາ ຈະ ໄດ້ ຮັບ ຄວາມ ຖ່ອມ ຕົນ ເພື່ອ ຈະ ໄດ້ ຮັບ.
ໃນຄວາມເປັນຈິງ, ສັງຄົມໄດ້ຂ້າຕົວຕາຍ.
12 ບັດ ນີ້ ຫລາຍ ຄົນ ບໍ່ ໄດ້ ຕາຍ ໄປ, ແຕ່ ລູກ ກັບ ເມຍ ຂອງ ຕົນ ອີກ, ແຕ່ ເຂົາ ໄດ້ ຕາຍ ໄປ ດ້ວຍ ຄວາມ ຕາຍ.


In [33]:
# Desubword the target file (reference) of the test dataset
# Note: You might as well have split files *before* subwording during dataset preperation, 
# but sometimes datasets have tokeniztion issues, so this way you are sure the file is really untokenized.
!python3 MT-Preparation/subwording/3-desubword.py /kaggle/input/nmt-data-subword/target.model /kaggle/input/nmt-data-subword/split_val2023_10kto100k.lo.subword

Traceback (most recent call last):
  File "/kaggle/working/MT-Preparation/subwording/3-desubword.py", line 21, in <module>
    with open(target_pred) as pred, open(target_decodeded, "w+") as pred_decoded:
OSError: [Errno 30] Read-only file system: '/kaggle/input/nmt-data-subword/split_val2023_10kto100k.lo.subword.desubword'


In [18]:
# Check the first 5 lines of the desubworded reference
!head -n 5 split_val2023_10kto100k.lo.subword.desubword

head: cannot open 'split_val2023_10kto100k.lo.subword.desubword' for reading: No such file or directory


# MT Evaluation

There are several MT Evaluation metrics such as BLEU, TER, METEOR, COMET, BERTScore, among others.

Here we are using BLEU. Files must be detokenized/desubworded beforehand.

In [29]:
# Download the BLEU script
!wget https://raw.githubusercontent.com/ymoslem/MT-Evaluation/main/BLEU/compute-bleu.py

--2023-11-08 06:52:26--  https://raw.githubusercontent.com/ymoslem/MT-Evaluation/main/BLEU/compute-bleu.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 957 [text/plain]
Saving to: ‘compute-bleu.py.1’

compute-bleu.py.1   100%[===================>]     957  --.-KB/s    in 0s      

2023-11-08 06:52:26 (67.7 MB/s) - ‘compute-bleu.py.1’ saved [957/957]



In [31]:
# Install sacrebleu
!pip3 install sacrebleu

In [36]:
# Evaluate the translation (without subwording)
!python3 compute-bleu.py /kaggle/input/valtest-nmt/split_val2023_10kto100k.lo UN.lo.translated.desubword

Reference 1st sentence: ພ້ອມກັນນັ້ນ, ຍູ້ແຮງການຮ່ວມມືຕໍ່ສູ້ປ້ອງກັນ, ຕ້ານອາດຊະຍາກໍາ; ເພີ່ມທະວີການຮ່ວມມືບໍາລຸງສ້າງ ລະຫວ່າງ 2 ຝ່າຍ ເພື່ອແນໃສ່ຍົກສູງກໍາລັງຄວາມສາມາດໃຫ້ພະນັກງານນັກຮົບໃນກະຊວງຕໍາຫຼວດ ຫວຽດນາມ, ໂດຍສະເພາະແມ່ນໃນຂົງເຂດປ້ອງກັນ, ຕ້ານການອາດຊະຍາກໍາ, ເຕັກນິກທາງອາຍາ, ຄວາມປອດໄພໃນການຈະລາຈອນ.
MTed 1st sentence: ພ້ອມກັນນັ້ນ, ກະຊວງກະສິກໍາ ແລະ ພັດທະນາຊົນນະບົດ ຫວຽດນາມ ກໍ່ເພີ່ມທະວີວຽກງານຈັດຕັ້ງການຮັບທາບ, ພິເສດແມ່ນ ພາຍ 2 ປະເທດ, ພິເສດແມ່ນພະຍຸ, ປ້ອງກັນ, ຕ້ານບັນດາຂໍ້ກໍານົດ, ຕ້ານເຊື້ອພະຍາດຕິດຕໍ່, ພິເສດແມ່ນວິສາຫະກິດ...
BLEU:  15.905601286465224


# More Features and Directions to Explore

Experiment with the following ideas:
* Icrease `train_steps` and see to what extent new checkpoints provide better translation, in terms of both BLEU and your human evaluation.

* Check other MT Evaluation mentrics other than BLEU such as [TER](https://github.com/mjpost/sacrebleu#ter), [WER](https://blog.machinetranslation.io/compute-wer-score/), [METEOR](https://blog.machinetranslation.io/compute-bleu-score/#meteor), [COMET](https://github.com/Unbabel/COMET), and [BERTScore](https://github.com/Tiiiger/bert_score). What are the conceptual differences between them? Is there special cases for using a specific metric?

* Continue training from the last model checkpoint using the `-train_from` option, only if the training stopped and you want to continue it. In this case, `train_steps` in the config file should be larger than the steps of the last checkpoint you train from.
```
!onmt_train -config config.yaml -train_from models/model.fren_step_3000.pt
```

* **Ensemble Decoding:** During translation, instead of adding one model/checkpoint to the `-model` argument, add multiple checkpoints. For example, try the two last checkpoints. Does it improve quality of translation? Does it affect translation seepd?

* **Averaging Models:** Try to average multiple models into one model using the [average_models.py](https://github.com/OpenNMT/OpenNMT-py/blob/master/onmt/bin/average_models.py) script, and see how this affects translation quality.
```
python3 average_models.py -models model_step_xxx.pt model_step_yyy.pt -output model_avg.pt
```
* **Release the model:** Try this command and see how it reduce the model size.
```
onmt_release_model --model "model.pt" --output "model_released.pt
```
* **Use CTranslate2:** For efficient translation, consider using [CTranslate2](https://github.com/OpenNMT/CTranslate2), a fast inference engine. Check out an [example](https://gist.github.com/ymoslem/60e1d1dc44fe006f67e130b6ad703c4b).

* **Work on low-resource languages:** Find out more details about [how to train NMT models for low-resource languages](https://blog.machinetranslation.io/low-resource-nmt/).

* **Train a multilingual model:** Find out helpful notes about [training multilingual models](https://blog.machinetranslation.io/multilingual-nmt).

* **Publish a demo:** Show off your work through a [simple demo with CTranslate2 and Streamlit](https://blog.machinetranslation.io/nmt-web-interface/).
